In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv("./db/lithium-reciclying-plants.csv")
df[['lon', 'lat']] = df['WKT'].str.extract(r'POINT \(([^ ]+) ([^)]+)\)').astype(float)


In [4]:
df

,WKT,name,description,lat,lon
0,POINT (-77.67829119999999 43.1981518),Li-Cycle New York Spoke,Expects to use a hydrometallurgical process an...,-77.678291,43.198152
1,POINT (-111.7696764 32.87824409999999),Quemetco Metals Ltd Inc,Quemetco Metals Ltd Inc - Ecobat. It will ini...,-111.769676,32.878244
2,POINT (-95.46281119999999 29.7386312),ACE Green Recycling,https://www.acegreenrecycling.com/ It plans t...,-95.462811,29.738631
3,POINT (-119.7414856 39.204376),Redwood Materials,"Unspecified capacity, up to 95% material recov...",-119.741486,39.204376
4,POINT (-119.8217446 39.5242032),American Battery Technology Company,"https://americanbatterytechnology.com 20,000 t...",-119.821745,39.524203
5,POINT (-119.4399422 39.53570729999999),Tesla Giga Nevada,"Tesla Gigafactory 1, located in Sparks, Nevada...",-119.439942,39.535707
6,POINT (-83.84230749999999 33.6122481),Ascend Elements,"Inc.,Unspecified capacity, recycling process ...",-83.842307,33.612248
